In [1]:
from ipywidgets import interact, widgets, Label
import nbinteract as nbi
import numpy as np

class trajectory(object):
    def __init__(self):
        self.xpts = np.arange(0)
        self.ypts = np.arange(0) 
        
    def xvals(self, coef, m, wind, angle, velocity):
        '''
        DESCRIPTION:
        Calculates the x- and y- coordinates for a trajectory through the air
        under the influences of gravity, air resistance, and horizontal wind.
        
        PARAMETERS:
        coef -- (float) A constant representing the characterstics of the
                        projectile and the fluid through which it travels.
        m -- (float) Mass in [kg].
        wind -- (float) Horizontal wind velocity relative to ground [m/s].
        angle -- (float) The launch angle in [degrees].
        velocity -- (float) Launch speed in [m/s].
        '''
        self.xpts = np.arange(0)
        self.ypts = np.arange(0) 
        dt = 0.1
        g = -9.8
        x = 0
        y = 0
        v = velocity
        vx = v * np.cos(np.radians(angle))
        vy = v * np.sin(np.radians(angle))
        v2 = (vx - wind) ** 2 + vy ** 2
        a = coef * v2 / m
        # use angle relative to wind to find a-components.
        angle = np.arctan(vy / np.abs(vx - wind))
        ax = a * np.cos(angle) * (-(vx - wind) / np.abs(vx - wind + 1e-8))
        ay = a * np.sin(angle) * (-vy / np.abs(vy)) + g

        while True:
            # This is not the true angle, but rather the angle relative to the wind.
            angle = np.arctan(vy / np.abs(vx - wind))
            v2 = (vx - wind) ** 2 + vy ** 2
            a = coef * v2 / m
            ax = a * np.cos(angle) * (-(vx - wind) / np.abs(vx - wind + 1e-8))
            ay = a * np.sin(angle) * (-vy / np.abs(vy + 1e-8)) + g
            vx += ax * dt
            vy += ay * dt         
#             x += vx * dt + 0.5 * ax * dt ** 2
#             y += vy * dt + 0.5 * ay * dt ** 2
            x += vx*dt
            y += vy*dt

            if (y <= 0):
                break
            else:
                self.xpts = np.append(self.xpts, x)
                self.ypts = np.append(self.ypts, y)

        return self.xpts
    
    def yvals(self, foo):
        return self.ypts
    
    
a = trajectory()
coefficient = widgets.BoundedFloatText(value=0, min=0, step=0.0001, description=r'\(\frac{1}{2}C_dA\rho\)')
mass = widgets.BoundedFloatText(value=1, min=0.000001, description='mass [kg]')
xmax = 500
ymax = 500
opts = {'title': 'Projectile Trajectory',
        'xlim': (0,xmax),
        'ylim': (0,ymax),
        'xlabel': 'Horizontal Displacement [m]',
        'ylabel': 'Vertical Displacement [m]'}

nbi.line(a.xvals, a.yvals, coef=coefficient, m=mass, wind=(-25,25),
         angle=(1,89), velocity=(1,100), options=opts)